# In this notebook I plan on loading in my data, structuring it and getting it ready for EDA, there will be no EDA in this notebook

In [24]:
import os # in case I need to do anything with path/dir
import ast

import numpy as np
import pandas as pd

from glob import glob

### Load in the data using glob

In [2]:
# use glob to get all of the csv files
csv_files = glob("./zippedData/*.csv")
csv_files

['./zippedData/tmdb.movies.csv',
 './zippedData/imdb.title.crew.csv',
 './zippedData/tn.movie_budgets.csv',
 './zippedData/imdb.title.ratings.csv',
 './zippedData/imdb.name.basics.csv',
 './zippedData/imdb.title.principals.csv',
 './zippedData/imdb.title.akas.csv',
 './zippedData/bom.movie_gross.csv',
 './zippedData/imdb.title.basics.csv']

In [6]:
movie_budgets = pd.read_csv("zippedData/tn.movie_budgets.csv")
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


In [7]:
movie_info = pd.read_csv("zippedData/tmdb.movies.csv")
movie_info.head()

,Unnamed: 0,genre_ids,id,original_language,original_title,popularity,release_date,title,vote_average,vote_count
0,0,"[12, 14, 10751]",12444,en,Harry Potter and the Deathly Hallows: Part 1,33.533,2010-11-19,Harry Potter and the Deathly Hallows: Part 1,7.7,10788
1,1,"[14, 12, 16, 10751]",10191,en,How to Train Your Dragon,28.734,2010-03-26,How to Train Your Dragon,7.7,7610
2,2,"[12, 28, 878]",10138,en,Iron Man 2,28.515,2010-05-07,Iron Man 2,6.8,12368
3,3,"[16, 35, 10751]",862,en,Toy Story,28.005,1995-11-22,Toy Story,7.9,10174
4,4,"[28, 878, 12]",27205,en,Inception,27.920,2010-07-16,Inception,8.3,22186


In [20]:
def get_genres(movie_title):
    try:
        genres = movie_info.loc[movie_info['title']==movie_title, 'genre_ids'].values[0]
    except:
        genres = None
    return genres

In [21]:
get_genres(movie_budgets['movie'][0])

'[28, 12, 14, 878]'

In [22]:
movie_budgets['genres'] = movie_budgets['movie'].apply(get_genres)

In [23]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,genres
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279","[28, 12, 14, 878]"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875","[12, 28, 14]"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",None
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963","[28, 12, 878]"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",None


In [28]:
def convert_string_list(string):
    try:
        return ast.literal_eval(string)
    except:
        return None

movie_budgets['genres'] = movie_budgets['genres'].apply(convert_string_list)

In [29]:
movie_budgets.head()

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross,genres
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279","[28, 12, 14, 878]"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875","[12, 28, 14]"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350",None
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963","[28, 12, 878]"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747",None


In [30]:
movie_budgets.to_csv("zippedData/movie_budgets_with_genres.csv", index=False)

# Loads in all files as dataframes into a dictionary

In [62]:
# create a dictionary with 
# keys as csv filenames (cleaned)
# values as their respective dataframes
csv_files_dict = {}
for filename in csv_files:
    filename_cleaned = os.path.basename(filename).replace(".csv", "").replace(".", "_") # cleaning the filenames
    filename_df = pd.read_csv(filename, index_col=0)
    csv_files_dict[filename_cleaned] = filename_df # use cleanfilename as a key and the df as its value
    

In [67]:
csv_files_dict.keys()

dict_keys(['tmdb_movies', 'imdb_title_crew', 'tn_movie_budgets', 'imdb_title_ratings', 'imdb_name_basics', 'imdb_title_principals', 'imdb_title_akas', 'bom_movie_gross', 'imdb_title_basics'])

### Load files into sqlite database

In [63]:
import sqlite3

In [65]:
# this will create a sqlite file in your directory
conn = sqlite3.connect("movies_db.sqlite")

# One-off example, just showing you how to convert a dataframe into a sqlite table

In [ ]:
# this converts a dataframe to a sqlite table, with the ame 'tmdb_movies'
# just let sql create the schema itself

df_tmdb_movies.to_sql("tmdb_movies", conn)

# A function to create a sqlite table from a dataframe

In [59]:
# Write a function that converts the dataframe to a sqlite table
def create_sql_table_from_df(df, name, conn):
    # Use try except
    # it will try to make a table
    # if a table exists the function will execute whatever you put in the except part
    try:
        df.to_sql(name, conn)
        print(f"Created table {name}")
    
    # if the table exists t will tell you, and won't cause an error
    except Exception as e:
        print(f"could not make table {name}")
        print(e)

In [68]:
# Looping through the dictionary from earlier, that contains the keys and dataframes of all the files
# We can create the tables programmatically


for name, table in csv_files_dict.items():
    create_sql_table_from_df(table, name, conn)

Created table tmdb_movies
Created table imdb_title_crew
Created table tn_movie_budgets
Created table imdb_title_ratings
Created table imdb_name_basics
Created table imdb_title_principals
Created table imdb_title_akas
Created table bom_movie_gross
Created table imdb_title_basics


# Add your sqlite DB to a .gitignore file otherwise you can't push it to github

In [47]:
# Now create a .gitignore file that will ignore the files that you unzipped and the sqlite file
# look at mine for a reference
# you cannot push files bigger than 100 MB to github from your computer

# you can create your file here in jupyter and open it from jupyter
with open("./.gitignore", "w+") as f:
    f.write("*.sqlite") # put files you want to ignore here
    f.write("\n") # insert a new line after each file
    f.write("zippedData/")
    f.write("\n")
    f.write("zippedData/*.csv")
    f.write("\n")
    f.write("zippedData/*.gz")

In [69]:
# view all tables in db
conn.execute("select name from sqlite_master where type='table';").fetchall()

[('tmdb_movies',),
 ('imdb_title_crew',),
 ('tn_movie_budgets',),
 ('imdb_title_ratings',),
 ('imdb_name_basics',),
 ('imdb_title_principals',),
 ('imdb_title_akas',),
 ('bom_movie_gross',),
 ('imdb_title_basics',)]

# Now continue doing all the EDA work in the EDA notebook (you have to make this yourself)